# FASE DE EXPLORACIÓN Y ANÁLISIS DE LOS DATOS

## 1.) IMPORTACIÓN DE LIBRERIAS

In [25]:
import pandas as pd
import numpy as np
import os
import re

## 2.) Configuración Inicial y Funciones de Utilidad

In [26]:
# Configuración inicial y funciones auxiliares
def normalizar_texto(texto):
    """Normaliza texto para comparaciones más robustas"""
    if pd.isna(texto):
        return ""
    return str(texto).lower().strip()

def cargar_y_combinar_datos(archivos):
    """Carga y combina todos los archivos CSV en un solo DataFrame"""
    dfs = []
    for archivo in archivos:
        if os.path.exists(archivo):
            encodings = ["latin-1", "ISO-8859-1", "utf-8"]
            separadores = [";", ","]

            cargado = False
            for encoding in encodings:
                if cargado:
                    break
                for sep in separadores:
                    try:
                        df = pd.read_csv(archivo, encoding=encoding, sep=sep)
                        if len(df.columns) > 1:
                            dfs.append(df)
                            print(f"✓ {archivo} cargado con encoding {encoding} y separador '{sep}'")
                            cargado = True
                            break
                    except (UnicodeDecodeError, pd.errors.ParserError):
                        continue
            if not cargado:
                print(f"✗ No se pudo cargar {archivo} con ninguna combinación de encoding/separador")
        else:
            print(f"Advertencia: No se encontró el archivo {archivo}")

    if not dfs:
        raise Exception("No se pudieron cargar ninguno de los archivos")

    return pd.concat(dfs, ignore_index=True)

##Definimos funciones básicas para normalizar texto y cargar múltiples archivos CSV con diferentes codificaciones y separadores, manejando posibles errores de lectura.



## 3.)  Identificación Automática de Columnas

In [27]:
# Función para identificar automáticamente las columnas relevantes
def identificar_columnas(df):
    """Identifica automáticamente los nombres de las columnas clave"""
    columnas = df.columns.tolist()
    print("Columnas disponibles:", columnas)

    # Buscar posibles nombres para documento
    doc_posibles = ["Documento", "documento", "DOCUMENTO", "NumeroDocumento", "NroDocumento"]
    for doc in doc_posibles:
        if doc in columnas:
            doc_col = doc
            break
    else:
        for col in columnas:
            if "doc" in col.lower() or "id" in col.lower() or "numero" in col.lower():
                doc_col = col
                break
        else:
            doc_col = columnas[1] if len(columnas) > 1 else columnas[0]

    # Buscar posibles nombres para medicamento
    med_posibles = ["Medicamento", "medicamento", "MEDICAMENTO", "MedicamentoNombre", "NombreMedicamento"]
    for med in med_posibles:
        if med in columnas:
            med_col = med
            break
    else:
        for col in columnas:
            if "medic" in col.lower() or "drug" in col.lower() or "farma" in col.lower():
                med_col = col
                break
        else:
            med_col = columnas[4] if len(columnas) > 4 else columnas[2]

    print(f"Usando columna para documento: {doc_col}")
    print(f"Usando columna para medicamento: {med_col}")

    return doc_col, med_col

## Esta función identifica automáticamente las columnas de documento y medicamento en el dataset, lo que hace el script más robusto frente a variaciones en los nombres de columnas.

## 4.)  Detección de Medicamentos de Interés

In [28]:
# Funciones para detectar medicamentos de los 7 grupos de interés
def buscar_medicamentos_exactos(texto_medicamento):
    """Busca EXACTAMENTE los medicamentos de interés en el texto, evitando falsos positivos"""
    texto = normalizar_texto(texto_medicamento)

    # Diccionario de medicamentos con sus nombres EXACTOS y grupos
    medicamentos_exactos = {
        # GRUPO ESPECIAL
        "metoprolol": "GE_metoprolol",
        "propranolol": "GE_metoprolol",
        "propanolol": "GE_metoprolol",
        "hidroclorotiazida": "GE_hidroclorotiazida",
        # ARA II
        "irbesartan": "ARA_II",
        "valsartan": "ARA_II",
        "olmesartan": "ARA_II",
        "telmisartan": "ARA_II",
        "losartan": "ARA_II",
        # IECA
        "enalapril": "IECA",
        "captopril": "IECA",
        "perindopril": "IECA",
        # Calcioantagonistas
        "amlodipino": "Calcioantagonistas",
        "nifedipino": "Calcioantagonistas",
        "verapamilo": "Calcioantagonistas",
        # Otros diuréticos
        "espironolactona": "Otros_diuréticos",
        "furosemida": "Otros_diuréticos",
        "indapamida": "Otros_diuréticos",
        "clortalidona": "Otros_diuréticos",
        # Otros Beta-Bloqueadores
        "bisoprolol": "Otros_Beta_Bloqueadores",
        "carvedilol": "Otros_Beta_Bloqueadores",
        "nebivolol": "Otros_Beta_Bloqueadores",
        # Otros antihipertensivos
        "minoxidil": "Otros_antihipertensivos",
        "prazosina": "Otros_antihipertensivos",
        "clonidina": "Otros_antihipertensivos",
    }

    grupos_encontrados = set()

    # Buscar cada medicamento EXACTAMENTE en el texto
    for medicamento, grupo in medicamentos_exactos.items():
        # Usar regex para buscar la palabra completa, evitando subcadenas
        if re.search(r"\b" + re.escape(medicamento) + r"\b", texto):
            grupos_encontrados.add(grupo)

    return list(grupos_encontrados)

## Explicación: Define la lógica para detectar exactamente los medicamentos de los 7 grupos de interés usando búsqueda por palabras completas para evitar falsos positivos.

## 5.) Detección Inteligente de Medicamentos "X"

In [29]:
# Función para detectar medicamentos adicionales (X) de forma inteligente
def tiene_medicamento_x(texto_medicamento, grupos_encontrados):
    """Determina si hay medicamentos X adicionales de manera más inteligente"""
    texto = normalizar_texto(texto_medicamento)

    # Lista de palabras comunes en descripciones de medicamentos (NO son medicamentos X)
    palabras_no_x = {
        "mg", "mcg", "g", "ml", "l", "cc", "tableta", "tabletas", "comprimido", "comprimidos",
        "capsula", "capsulas", "cápsula", "cápsulas", "gragea", "grageas", "inyección", "ampolla",
        "frasco", "sobre", "suspension", "suspensión", "jarabe", "crema", "pomada", "supositorio",
        "spray", "inhalador", "parche", "ungüento", "oral", "intramuscular", "intravenoso",
        "subcutaneo", "subcutáneo", "topico", "tópico", "rectal", "vaginal", "oftalmico", "oftálmico",
        "otico", "ótico", "nasal", "cada", "horas", "día", "dias", "semana", "semanas", "mes", "meses",
        "año", "años", "dosis", "frecuencia", "tratamiento", "tomar", "aplicar", "uso", "adultos",
        "niños", "pacientes", "administrar", "via", "cad", "diaria", "semanal", "mensual", "anual",
        "continuo", "alternos", "mg", "mcg", "g", "ml", "l", "cc", "ui", "unidad", "unidades",
        "por", "con", "sin", "de", "la", "el", "y", "o", "para", "entre", "hasta", "desde", "sobre",
        "bajo", "tras", "durante", "antes", "después", "al", "del", "se", "es", "en", "a", "u", "un",
        "una", "unos", "unas", "uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve",
        "diez", "cien", "mil", "medio", "media", "cuarto", "cuarta", "primera", "segunda", "tercera",
        "tartrato", "bloqueadores", "antihipertensivos", "diuréticos", "bloqueador", "antihipertensivo",
        "diurético", "calcioantagonistas", "calcioantagonista", "hidroclorotiazida", "clorhidrato",
        "maleato", "succinato", "besilato", "valsartan", "irbesartan", "telmisartan", "olmesartan",
        "losartan", "enalapril", "captopril", "perindopril", "amlodipino", "nifedipino", "verapamilo",
        "espironolactona", "furosemida", "indapamida", "clortalidona", "bisoprolol", "carvedilol",
        "nebivolol", "minoxidil", "prazosina", "clonidina", "metoprolol", "propranolol", "propanolol",
    }

    # Remover los medicamentos encontrados
    texto_limpio = texto
    for grupo in grupos_encontrados:
        medicamentos_grupo = {
            "GE_metoprolol": ["metoprolol", "propranolol", "propanolol"],
            "GE_hidroclorotiazida": ["hidroclorotiazida"],
            "ARA_II": ["irbesartan", "valsartan", "olmesartan", "telmisartan", "losartan"],
            "IECA": ["enalapril", "captopril", "perindopril"],
            "Calcioantagonistas": ["amlodipino", "nifedipino", "verapamilo"],
            "Otros_diuréticos": ["espironolactona", "furosemida", "indapamida", "clortalidona"],
            "Otros_Beta_Bloqueadores": ["bisoprolol", "carvedilol", "nebivolol"],
            "Otros_antihipertensivos": ["minoxidil", "prazosina", "clonidina"],
        }

        if grupo in medicamentos_grupo:
            for med in medicamentos_grupo[grupo]:
                texto_limpio = re.sub(r"\b" + re.escape(med) + r"\b", "", texto_limpio)

    # Remover palabras no-X y números, buscar palabras de 4+ letras
    palabras = re.findall(r"[a-zA-Z]{4,}", texto_limpio)  # Solo palabras de 4+ letras
    palabras_filtradas = [p for p in palabras if p not in palabras_no_x]

    # También verificar si hay signos de múltiples medicamentos (+, &, "y", "con")
    tiene_multiples = bool(re.search(r"\+|\&| y | con |/\s*[a-zA-Z]", texto))

    return len(palabras_filtradas) > 0 or tiene_multiples

## Explicación: Implementa la lógica inteligente para detectar medicamentos "X" (cualquier medicamento adicional), filtrando palabras comunes de dosis y formas farmacéuticas para evitar falsos positivos.

## 6.) Sistema de Categorización por Registro

In [30]:
## # Sistema completo de categorización por registro individual
def determinar_categorizacion_por_registro(medicamento_str):
    """Determina la categorización INDIVIDUAL para CADA REGISTRO"""
    # Buscar grupos de medicamentos EXACTOS
    grupos = buscar_medicamentos_exactos(medicamento_str)

    # Si no hay medicamentos de interés, no procesar
    if not grupos:
        return "NO_APLICA"

    # Determinar si hay medicamento X
    hay_x = tiene_medicamento_x(medicamento_str, grupos)

    # Mapeo de nombres de grupos a nombres de categorización
    mapeo_categorias = {
        "GE_metoprolol": "GE metoprolol",
        "GE_hidroclorotiazida": "GE Hidroclorotiazida",
        "ARA_II": "ARA II",
        "IECA": "IECA",
        "Calcioantagonistas": "Calcioantagonistas",
        "Otros_diuréticos": "Otros diuréticos",
        "Otros_Beta_Bloqueadores": "Otros Beta-Bloqueadores",
        "Otros_antihipertensivos": "Otros Anti-Hipertensivos",
    }

    # Convertir grupos internos a nombres de categoría
    grupos_categoria = [mapeo_categorias[grupo] for grupo in grupos]

    # REGLAS DE CATEGORIZACIÓN - POR REGISTRO INDIVIDUAL

    # 1. Verificar presencia de Grupo Especial (GE)
    tiene_ge_metoprolol = "GE metoprolol" in grupos_categoria
    tiene_ge_hidro = "GE Hidroclorotiazida" in grupos_categoria

    # Contar medicamentos de grupos no especiales en ESTE registro
    grupos_no_especiales = [
        g
        for g in grupos_categoria
        if g not in ["GE metoprolol", "GE Hidroclorotiazida"]
    ]
    total_grupos_no_especiales = len(grupos_no_especiales)

    # Si hay Grupo Especial en este registro
    if tiene_ge_metoprolol or tiene_ge_hidro:
        # CASO 1: Solo metoprolol (sin hidro, sin otros grupos, sin X)
        if (
                tiene_ge_metoprolol
                and not tiene_ge_hidro
                and total_grupos_no_especiales == 0
                and not hay_x
        ):
            return "GE metoprolol univ"

        # CASO 2: Solo hidroclorotiazida (sin metoprolol, sin otros grupos, sin X)
        elif (
                not tiene_ge_metoprolol
                and tiene_ge_hidro
                and total_grupos_no_especiales == 0
                and not hay_x
        ):
            return "GE Hidroclorotiazida univ"

        # CASO 3: Metoprolol + hidroclorotiazida (sin otros grupos, sin X)
        elif (
                tiene_ge_metoprolol
                and tiene_ge_hidro
                and total_grupos_no_especiales == 0
                and not hay_x
        ):
            return "GE Hidro-Meto univ"

        # CASO 4: Metoprolol + X (CUALQUIER medicamento adicional)
        elif tiene_ge_metoprolol and not tiene_ge_hidro and hay_x:
            return "GE Meto-X univ"

        # CASO 5: Hidroclorotiazida + X (CUALQUIER medicamento adicional)
        elif not tiene_ge_metoprolol and tiene_ge_hidro and hay_x:
            return "GE Hidro-X univ"

        # CASO 6: Metoprolol + hidroclorotiazida + X (CUALQUIER medicamento adicional)
        elif tiene_ge_metoprolol and tiene_ge_hidro and hay_x:
            return "GE Hidro-Meto 2"

        # CASO 7: Metoprolol + otros grupos (sin X)
        elif (
                tiene_ge_metoprolol
                and not tiene_ge_hidro
                and total_grupos_no_especiales > 0
                and not hay_x
        ):
            otros_grupos_str = " & ".join(grupos_no_especiales)
            return f"GE Meto & {otros_grupos_str} univ"

        # CASO 8: Hidroclorotiazida + otros grupos (sin X)
        elif (
                not tiene_ge_metoprolol
                and tiene_ge_hidro
                and total_grupos_no_especiales > 0
                and not hay_x
        ):
            otros_grupos_str = " & ".join(grupos_no_especiales)
            return f"GE Hidro & {otros_grupos_str} univ"

    # 2. Si no hay Grupo Especial, categorizar grupos no especiales
    else:
        if len(grupos_categoria) == 1:
            return f"{grupos_categoria[0]} univ"
        else:
            # Múltiples grupos no especiales
            return f"{' & '.join(grupos_categoria)} univ"

    return "Categorización no definida"

def es_registro_de_interes(medicamento_str):
    """Determina si un registro contiene al menos un medicamento de interés"""
    grupos = buscar_medicamentos_exactos(medicamento_str)
    return len(grupos) > 0



## Implementa el sistema completo de categorización que aplica las reglas de negocio para asignar la etiqueta correcta a cada registro individualmente.

## 7.) Procesamiento Principal y Generación del CSV Final

In [31]:
# Procesamiento principal y generación del dataset final
def procesar_csv_por_registro(df, doc_col, med_col):
    """Procesa CADA REGISTRO individualmente y genera el CSV de salida"""
    registros_procesados = []

    print(f"Procesando {len(df)} registros individualmente...")
    registros_con_interes = 0

    for indice, fila in df.iterrows():
        medicamento = fila[med_col]

        # SOLO procesar si el registro tiene medicamentos de interés
        if es_registro_de_interes(medicamento):
            registros_con_interes += 1

            # Crear copia del registro
            registro_procesado = fila.to_dict()

            # Asignar categorización INDIVIDUAL para este registro
            categoria = determinar_categorizacion_por_registro(medicamento)
            registro_procesado["Categorización"] = categoria

            registros_procesados.append(registro_procesado)

    print(f"Registros con medicamentos de interés: {registros_con_interes}")
    return registros_procesados

## 8.) Ejecución Completa del Pipeline

In [32]:
# Ejecución completa del pipeline de procesamiento
def main():
    """Función principal"""
    archivos = [
        "Antihipertensivos1.csv",
        "Antihipertensivos2.csv",
        "OtrosMedicamentos.csv",
    ]

    try:
        print("Cargando y combinando datos desde CSV...")
        df_completo = cargar_y_combinar_datos(archivos)

        print("\nIdentificando columnas...")
        doc_col, med_col = identificar_columnas(df_completo)

        print("Procesando registros INDIVIDUALMENTE...")
        registros_finales = procesar_csv_por_registro(df_completo, doc_col, med_col)

        # Crear DataFrame final
        if registros_finales:
            df_final = pd.DataFrame(registros_finales)

            # Reordenar columnas para que 'Categorización' esté al final
            if "Categorización" in df_final.columns:
                columnas = [
                               col for col in df_final.columns if col != "Categorización"
                           ] + ["Categorización"]
                df_final = df_final[columnas]

            # Guardar el archivo final
            archivo_salida = "registros_clasificados_por_medicamento_final.csv"
            df_final.to_csv(archivo_salida, index=False, encoding="utf-8-sig", sep=";")

            print(f"\n=== PROCESO COMPLETADO CON ÉXITO ===")
            print(f"Archivo generado: {archivo_salida}")
            print(f"Total de registros procesados: {len(df_final)}")

            # Mostrar resumen de categorizaciones
            print(f"\nResumen de categorizaciones POR REGISTRO:")
            categorias_count = df_final["Categorización"].value_counts()
            for categoria, count in categorias_count.items():
                print(f"  {categoria}: {count} registros")

            print(f"\nTotal de categorías únicas: {len(categorias_count)}")

        else:
            print("No se encontraron registros con medicamentos de interés")

    except Exception as e:
        print(f"Error durante la ejecución: {str(e)}")
        import traceback
        traceback.print_exc()

# Ejecutar el proceso completo
if __name__ == "__main__":
    main()

## Función principal que orquesta todo el proceso: carga de datos, identificación de columnas, procesamiento individual y generación del CSV final con estadísticas del proceso.

Cargando y combinando datos desde CSV...


C:\Users\Alejandro\AppData\Local\Temp\ipykernel_7924\961491581.py:22: DtypeWarning: Columns (13,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, encoding=encoding, sep=sep)
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_7924\961491581.py:22: DtypeWarning: Columns (13,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, encoding=encoding, sep=sep)


✓ Antihipertensivos1.csv cargado con encoding latin-1 y separador ';'
✓ Antihipertensivos2.csv cargado con encoding latin-1 y separador ';'


C:\Users\Alejandro\AppData\Local\Temp\ipykernel_7924\961491581.py:22: DtypeWarning: Columns (13,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, encoding=encoding, sep=sep)


✓ OtrosMedicamentos.csv cargado con encoding latin-1 y separador ';'

Identificando columnas...
Columnas disponibles: ['Secuencia', 'Documento', 'CodProced', 'Num_orden', 'Medicamento', 'Frecuencia', 'Dosis', 'Via', 'TTratamiento', 'Empresa', 'Cantidad', 'Item', 'Especialidad', 'NumProfe', 'MedicoOrdena', 'Nom1PAc', 'Nom2Pac', 'Apell1Pac', 'Apell2Pac', 'Fechnac', 'Sexo', 'Direcion', 'Tel', 'FechaOrden', 'CANTIDAD', 'Observaciones', 'Fecha_atencion', 'sede_id', 'lugar', 'num_formula', 'nombrepac', 'edad', 'telefono', 'sexo', 'direccion', 'empresa', 'dxppal', 'desdxppal']
Usando columna para documento: Documento
Usando columna para medicamento: Medicamento
Procesando registros INDIVIDUALMENTE...
Procesando 1020024 registros individualmente...


KeyboardInterrupt: 